In [2]:
import os
import pandas as pd
from local_python.general_utils import load_pd_from_json, number_to_string, check_duplicates

In [3]:
metric_name = "f1_macro"
column_names_groupby = [
    "architecture",
    "downstream",
    "pre-training",
    "number_of_samples",
    "model_name",
]
aggregating_functions = ["mean", "std", "count"]
path_metrics = "../runs/"
path_f1_scores_grouped = "../results/f1_scores.csv"
derma_datasets = ["DDI", "PAD-UFES-20", "HAM10000", "Fitzpatrick17k"]
plant_datasets = ["PlantDoc", "PlantDataset", "Cassava", "PlantVillage"]

name_map = {
    "Random": "Random",
    "ImageNet_1k_SL_V1": "ImageNet SL ",
    "ImageNet_1k_SL_WinKawaks": "ImageNet SL",
    "ImageNet_1k_SSL_SimCLR": "ImageNet SSL ",
    "ImageNet_1k_SSL_Dino": "ImageNet SSL",
    "ImageNet_AugReg": "ImageNet AR",
    "Derma_SSL_SimCLR": "Derma SSL ",
    "Derma": "Derma SSL",
    "PDDD": "Plant SL",
    "Plant": "Plant SSL",
}


In [4]:
def get_downstream(value):
    return get_feature_identifier_part(value, 0).replace("_", "-")


def get_architecture(value):
    return get_feature_identifier_part(value, 1)


def get_pretraining(value):
    return name_map[get_feature_identifier_part(value, 2)]


def get_feature_identifier_part(value, idx):
    return os.path.splitext(os.path.basename(value))[0].split("-")[idx]

def score_to_percent_string(values):
    (mean, std) = values
    mean_str = "%.1f" % (100 * mean)
    if 0==std:
        return mean_str
    std_str = "%.1f" % (100 * std)
    return f"{mean_str}±{std_str}"

In [5]:
df_master = pd.DataFrame()

for name in os.listdir(path_metrics):
    if name.endswith(".txt") and "metrics" in name:
        metric_file_path = os.path.join(path_metrics, name)
        df_full = load_pd_from_json(metric_file_path)
        df_full["number_of_samples"] = df_full["number_of_samples"].apply(
            number_to_string, "All"
        )
        df_full["architecture"] = df_full["feature_identifier"].apply(get_architecture)
        df_full["downstream"] = df_full["feature_identifier"].apply(get_downstream)
        df_full["pre-training"] = df_full["feature_identifier"].apply(get_pretraining)
        df_groups = df_full.groupby(column_names_groupby).agg(
            {metric_name: aggregating_functions}
        )
        df_master = pd.concat(
            [df_master, df_groups.reset_index()], ignore_index=True, axis=0
        )
print(f"{len(df_master)} rows in master file")

Read 7500 entries from Cassava-ResNet50-metrics.txt
Read 9000 entries from Cassava-ViT_T16-student-metrics.txt
Read 7500 entries from DDI-ResNet50-metrics.txt
Read 9000 entries from DDI-ViT_T16-student-metrics.txt
Read 7500 entries from Fitzpatrick17k-ResNet50-metrics.txt
Read 9000 entries from Fitzpatrick17k-ViT_T16-student-metrics.txt
Read 7500 entries from HAM10000-ResNet50-metrics.txt
Read 9000 entries from HAM10000-ViT_T16-student-metrics.txt
Read 3520 entries from master-metrics.txt
Read 6000 entries from PAD_UFES_20-ResNet50-metrics.txt
Read 7200 entries from PAD_UFES_20-ViT_T16-student-metrics.txt
Read 6000 entries from PlantDataset-ResNet50-metrics.txt
Read 7200 entries from PlantDataset-ViT_T16-student-metrics.txt
Read 6000 entries from PlantDoc-ResNet50-metrics.txt
Read 7200 entries from PlantDoc-ViT_T16-student-metrics.txt
Read 7500 entries from PlantVillage-ResNet50-metrics.txt
Read 9000 entries from PlantVillage-ViT_T16-student-metrics.txt
1485 rows in master file


In [6]:
for column_name in df_master.columns:
    unique_values = df_master[column_name].unique()
    if 20 < len(unique_values):
        print(f"{column_name} has {len(unique_values)} unique values")
    else:
        print(f"{column_name}: {unique_values}")

('architecture', ''): ['ResNet50' 'ViT_T16']
('downstream', ''): ['Cassava' 'DDI' 'Fitzpatrick17k' 'HAM10000' 'PAD-UFES-20' 'PlantDataset'
 'PlantDoc' 'PlantVillage']
('pre-training', ''): ['Derma SSL ' 'ImageNet SL ' 'ImageNet SSL ' 'Plant SL' 'Random'
 'Derma SSL' 'ImageNet AR' 'ImageNet SL' 'ImageNet SSL' 'Plant SSL']
('number_of_samples', ''): ['1' '10' '100' '3' '30' 'None']
('model_name', ''): ['dc' 'knn' 'lr']
('f1_macro', 'mean') has 1004 unique values
('f1_macro', 'std') has 989 unique values
('f1_macro', 'count'): [100  10  20]


In [7]:
df_master.columns = df_master.columns.map(lambda col: "_".join(col).strip("_"))
df_master

,architecture,downstream,pre-training,number_of_samples,model_name,f1_macro_mean,f1_macro_std,f1_macro_count
0,ResNet50,Cassava,Derma SSL,1,dc,0.019217,0.000000,100
1,ResNet50,Cassava,Derma SSL,1,knn,0.184686,0.035744,100
2,ResNet50,Cassava,Derma SSL,1,lr,0.163515,0.045919,100
3,ResNet50,Cassava,Derma SSL,10,dc,0.019217,0.000000,100
4,ResNet50,Cassava,Derma SSL,10,knn,0.208679,0.018469,100
...,...,...,...,...,...,...,...,...
1480,ViT_T16,PlantVillage,Random,3,knn,0.128046,0.010907,100
1481,ViT_T16,PlantVillage,Random,3,lr,0.175155,0.013696,100
1482,ViT_T16,PlantVillage,Random,30,dc,0.000604,0.000000,100
1483,ViT_T16,PlantVillage,Random,30,knn,0.205410,0.007056,100


In [8]:
df_master.to_csv(path_f1_scores_grouped)

In [9]:
column_order_model_name = [
    "dc",
    "lr",
    "knn",
]

len_before = len(df_master)

df_master = pd.read_csv(
    path_f1_scores_grouped,
    index_col=0,
    dtype={
        "pre-training": pd.CategoricalDtype(
            categories=list(name_map.values()), ordered=True
        ),
        "model_name": pd.CategoricalDtype(
            categories=column_order_model_name, ordered=True
        )
    },
)
assert len_before == len(df_master)
df_master.head(n=5)


,architecture,downstream,pre-training,number_of_samples,model_name,f1_macro_mean,f1_macro_std,f1_macro_count
0,ResNet50,Cassava,Derma SSL,1,dc,0.019217,0.000000,100
1,ResNet50,Cassava,Derma SSL,1,knn,0.184686,0.035744,100
2,ResNet50,Cassava,Derma SSL,1,lr,0.163515,0.045919,100
3,ResNet50,Cassava,Derma SSL,10,dc,0.019217,0.000000,100
4,ResNet50,Cassava,Derma SSL,10,knn,0.208679,0.018469,100


In [10]:
df_master["f1_macro"] = df_master[["f1_macro_mean", "f1_macro_std"]].apply(score_to_percent_string, axis=1)

In [11]:
def filter_and_unstack(df, filter_config):
    df_grouping_columns = [
        "downstream",
        "pre-training",
        "model_name",
    ]
    for filter_key, filter_list in filter_config.items():
        assert (
            filter_key in df.columns.values
        ), f"No column found with name {filter_key}"
        df = df[df[filter_key].isin(filter_list)]
    df = df[
        [
            "downstream",
            "pre-training",
            "model_name",
            "f1_macro",
        ]
    ]
    check_duplicates(df, df_grouping_columns)
    return df.set_index(df_grouping_columns).sort_index().unstack(level=0).unstack(level=-1).sort_index()

In [12]:
df_resnet = filter_and_unstack(df_master,
    {
        "number_of_samples": ["None"],
        "architecture": ["ResNet50"],
        "model_name": ["lr", "knn"],
    },
)
df_vit = filter_and_unstack(df_master,
    {
        "number_of_samples": ["None"],
        "architecture": ["ViT_T16"],
        "model_name": ["lr", "knn"],
    },
)
df_baseline = filter_and_unstack(df_master,
    {
        "number_of_samples": ["None"],
        "architecture": ["ResNet50"],
        "model_name": ["dc"],
    },
)
df_baseline.head(n=1)

f1_macro                                                         \
downstream    Cassava   DDI Fitzpatrick17k HAM10000 PAD-UFES-20 PlantDataset   
model_name         dc    dc             dc       dc          dc           dc   
pre-training                                                                   
Random           15.2  42.6           28.1     10.7         8.9          1.8   

                                    
downstream   PlantDoc PlantVillage  
model_name         dc           dc  
pre-training                        
Random            0.3          0.5

In [13]:
df_resnet["f1_macro"][derma_datasets].to_csv("../results/f1_scores_resnet_derma.csv")
df_resnet["f1_macro"][plant_datasets].to_csv("../results/f1_scores_resnet_plant.csv")
df_vit["f1_macro"][derma_datasets].to_csv("../results/f1_scores_vit_derma.csv")
df_vit["f1_macro"][plant_datasets].to_csv("../results/f1_scores_vit_plant.csv")

In [14]:
df_resnet

f1_macro                                               \
downstream      Cassava                 DDI           Fitzpatrick17k   
model_name           lr       knn        lr       knn             lr   
pre-training                                                           
Random         31.6±0.6  23.9±0.3  65.8±3.5  55.5±7.7       40.0±0.9   
ImageNet SL    62.3±0.5  49.1±0.9  63.5±4.8  60.0±2.9       58.5±0.8   
ImageNet SSL   21.1±0.5  22.8±0.6  52.3±4.2  55.8±5.9       35.6±0.5   
Derma SSL      16.6±0.1  27.8±0.8      42.6  51.1±2.9       29.7±0.1   
Plant SL       51.6±0.3  40.6±1.3  42.0±1.2  56.1±6.8       36.3±0.7   

                                                                   \
downstream               HAM10000           PAD-UFES-20             
model_name          knn        lr       knn          lr       knn   
pre-training                                                        
Random         41.4±0.5  33.6±1.2  22.1±1.2    33.4±2.0  19.7±2.2   
ImageNet SL    65.8±0.5  57.1±0.9  40.6±0.9    57.2±2.9  49.2±2.1   
ImageNet SSL   44.2±1.0  30.0±0.7  26.2±1.2    35.7±2.0  29.7±2.6   
Derma SSL      48.6±1.2  14.9±0.1  28.5±1.1    16.0±0.3  35.0±3.6   
Plant SL       51.1±0.7  31.2±0.6  32.8±0.5    35.4±2.9  34.6±3.8   

                                                                       \
downstream    PlantDataset            PlantDoc           PlantVillage   
model_name              lr       knn        lr       knn           lr   
pre-training                                                            
Random            34.6±1.1  17.2±2.2  13.5±1.9   6.7±1.9     70.5±0.1   
ImageNet SL       92.0±0.7  83.8±2.4  54.7±1.4  37.4±2.4     97.3±0.1   
ImageNet SSL      32.0±1.3  22.0±1.7  11.9±1.8   9.0±2.4     50.1±0.1   
Derma SSL          1.8±0.0  30.7±2.7   1.0±0.0  18.9±3.2      6.4±0.1   
Plant SL          67.4±0.7  60.5±1.5  65.3±1.7  41.3±1.5     99.1±0.1   

                         
downstream               
model_name          knn  
pre-training             
Random         39.9±0.5  
ImageNet SL    92.0±0.3  
ImageNet SSL   34.7±0.3  
Derma SSL      49.3±0.5  
Plant SL       98.7±0.2

In [20]:
df_master[
    (df_master["downstream"] == "PlantDataset") &
    (df_master["number_of_samples"] == "None") &
    (df_master["model_name"] == "lr")
]

,architecture,downstream,pre-training,number_of_samples,model_name,f1_macro_mean,f1_macro_std,f1_macro_count,f1_macro
737,ResNet50,PlantDataset,Derma SSL,None,lr,0.018185,0.000070,10,1.8±0.0
740,ResNet50,PlantDataset,ImageNet SL,None,lr,0.919847,0.006969,10,92.0±0.7
743,ResNet50,PlantDataset,ImageNet SSL,None,lr,0.319557,0.012793,10,32.0±1.3
746,ResNet50,PlantDataset,Plant SL,None,lr,0.674007,0.007207,10,67.4±0.7
749,ResNet50,PlantDataset,Random,None,lr,0.345627,0.010539,10,34.6±1.1
872,ViT_T16,PlantDataset,Derma SSL,None,lr,0.804049,0.007397,10,80.4±0.7
875,ViT_T16,PlantDataset,ImageNet AR,None,lr,0.799036,0.014813,10,79.9±1.5
878,ViT_T16,PlantDataset,ImageNet SL,None,lr,0.892825,0.009328,10,89.3±0.9
881,ViT_T16,PlantDataset,ImageNet SSL,None,lr,0.935971,0.008230,10,93.6±0.8
884,ViT_T16,PlantDataset,Plant SSL,None,lr,0.866817,0.009192,10,86.7±0.9
